In [1]:
## Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from ast import literal_eval

In [2]:
DATASET_NAME = 'iTunes-Amazon'

In [3]:
class LSH :
  
    # Random limits, const
    __max_rand =  1000
    __min_rand = -1000
    
    # Constructor
    def __init__(self, k, L, embedding_size = 150):
        
        # Number of hash function
        self.k = k
        
        # Number of attempts
        self.L = L
        
        # Embedding length 
        self.embedding_size = embedding_size
        
        # Random matrices
        self.normalized_random_matrices = []
        
        for i in range(self.L):
            random_matrix = np.random.randint(self.__min_rand, self.__max_rand,(self.k, self.embedding_size));
            
            # Append normalized random matrices
            self.normalized_random_matrices.append(normalize(random_matrix, axis=1, norm='l1'))
        
    
    # Locality Sensitive hash function
    def locality_sensitive_hash(self, embedding, matrix_index):
        out = 0
      
        for h in self.normalized_random_matrices[matrix_index]:
            if (np.dot(h, embedding) >= 0):
                out = (out << 1) | 1
            else:
                out = (out << 1) | 0

        return out
      
    # Divide in buckets using L-th matrix
    def divide_in_buckets(self, embeddings, matrix_index):
        out = {}
        for embedding in embeddings:
            hash = self.locality_sensitive_hash(embedding, matrix_index)
            if (hash in out):
                out[hash].append(embedding)
            else:
                out[hash] = [embedding]
            
        return out  


In [4]:
test = LSH(k=5, L=2, embedding_size=10)
print(test.normalized_random_matrices)

print(test.locality_sensitive_hash([1,2,3,4,5,6,7,8,9,10], 1))

[array([[-0.03735359,  0.08547009,  0.25735992, -0.16365939,  0.22063944,
         0.00094967, -0.13105413,  0.02184236, -0.05413105, -0.02754036],
       [ 0.14997116,  0.06460296, -0.11497789, -0.09171313, -0.18900211,
        -0.17381273,  0.02672563, -0.10382619,  0.03999231, -0.04537589],
       [ 0.00693423,  0.10905652, -0.20025215, -0.07669679, -0.11767178,
         0.1468796 , -0.01596974,  0.04391679,  0.12922883,  0.15339357],
       [ 0.08685485, -0.14381159,  0.01954234, -0.13646234,  0.11307834,
        -0.06881577, -0.16201771,  0.02538834,  0.08117588,  0.16285285],
       [ 0.06984281, -0.13628261,  0.13903743,  0.13061092, -0.11181332,
         0.06125425,  0.12688381, -0.06984281,  0.02981689,  0.12461514]]), array([[ 0.01001478, -0.12641602, -0.0766705 , -0.05302906, -0.15777376,
        -0.16269906,  0.09752093, -0.06879002,  0.10884912,  0.13823674],
       [-0.06443639, -0.16216216, -0.02719183,  0.13266315, -0.10728411,
         0.11470007,  0.08816744,  0.13991

In [5]:
## almost equals
embeddings = [[1.2345,2,3,4,5,6,10.4,8,9,10],[1,2,3,4,5,6,7,8,9,10],[1,2,3,5,5,6,7,8,9,10]]
print(len(test.divide_in_buckets(embeddings, 1)))

## not equals
embeddings = np.random.randint(-10000, 10000,(10000, 10))
print(len(test.divide_in_buckets(embeddings, 1)))


2
32


In [6]:
## TEST BLOCKING PERFORMANCE
#    Basta che per ogni tupla vado a prendere la sua corrispondente, ne calcolo
#     i vari L hash e controllo che almeno uno sia uguale e incremento un
#     contatore. La precisione è contatore/numero di tuple controllate, giusto?
def performance_test(filtered_dataset, k, L, embedding_size):
    
    match_found = 0
    
    lsh = LSH(k, L, embedding_size)
    
    # for each elemt in dataset
    for index, row in filtered_dataset.iterrows():
        x_embedding = np.array(literal_eval(row['left_table']))
        y_embedding = np.array(literal_eval(row['right_table']))
          
        x_hashs = set()
        y_hashs = set()
        for i in range(L):
            x_hashs.add(lsh.locality_sensitive_hash(x_embedding, i))
            y_hashs.add(lsh.locality_sensitive_hash(y_embedding, i))
        
        if (len(set.intersection(x_hashs, y_hashs)) > 0):
            match_found += 1
  
    
    return match_found / len(filtered_dataset.index)

In [7]:
## Open dataset 
df = pd.read_csv('../../lsh-test-data/' + DATASET_NAME + '-embeddings.csv')

## Remove 0 labled
df = df[df.label == 1]

precision_max = 0
k_max = 0
L_max = 0
for k in range(30):
    for L in range(10):
        precision = performance_test(df, k + 1, L + 1, 150)
        print("K: {0}, L: {1}, Precision:{2}".format(k + 1, L + 1, precision))
        if (precision >= precision_max):
            precision_max = precision
            k_max = k + 1
            L_max = L + 1

print("Max precision: {0}, k: {1}, L: {2}".format(precision_max, k_max, L_max))

K: 1, L: 1, Precision:0.9629629629629629
K: 1, L: 2, Precision:1.0
K: 1, L: 3, Precision:1.0
K: 1, L: 4, Precision:1.0
K: 1, L: 5, Precision:1.0
K: 1, L: 6, Precision:1.0
K: 1, L: 7, Precision:1.0
K: 1, L: 8, Precision:1.0
K: 1, L: 9, Precision:1.0
K: 1, L: 10, Precision:1.0
K: 2, L: 1, Precision:1.0
K: 2, L: 2, Precision:1.0
K: 2, L: 3, Precision:1.0
K: 2, L: 4, Precision:1.0
K: 2, L: 5, Precision:1.0
K: 2, L: 6, Precision:1.0
K: 2, L: 7, Precision:1.0
K: 2, L: 8, Precision:1.0
K: 2, L: 9, Precision:1.0
K: 2, L: 10, Precision:1.0
K: 3, L: 1, Precision:0.7037037037037037
K: 3, L: 2, Precision:1.0
K: 3, L: 3, Precision:1.0
K: 3, L: 4, Precision:1.0
K: 3, L: 5, Precision:1.0
K: 3, L: 6, Precision:1.0
K: 3, L: 7, Precision:1.0
K: 3, L: 8, Precision:1.0
K: 3, L: 9, Precision:1.0
K: 3, L: 10, Precision:1.0
K: 4, L: 1, Precision:0.8888888888888888
K: 4, L: 2, Precision:0.9629629629629629
K: 4, L: 3, Precision:1.0
K: 4, L: 4, Precision:1.0
K: 4, L: 5, Precision:1.0
K: 4, L: 6, Precision:1.0
K

K: 23, L: 4, Precision:0.4444444444444444
K: 23, L: 5, Precision:0.6666666666666666
K: 23, L: 6, Precision:0.5925925925925926
K: 23, L: 7, Precision:0.5925925925925926
K: 23, L: 8, Precision:0.8148148148148148
K: 23, L: 9, Precision:0.6666666666666666
K: 23, L: 10, Precision:0.7407407407407407
K: 24, L: 1, Precision:0.18518518518518517
K: 24, L: 2, Precision:0.3333333333333333
K: 24, L: 3, Precision:0.5185185185185185
K: 24, L: 4, Precision:0.5185185185185185
K: 24, L: 5, Precision:0.4074074074074074
K: 24, L: 6, Precision:0.2222222222222222
K: 24, L: 7, Precision:0.7407407407407407
K: 24, L: 8, Precision:0.5555555555555556
K: 24, L: 9, Precision:0.5555555555555556
K: 24, L: 10, Precision:0.7037037037037037
K: 25, L: 1, Precision:0.07407407407407407
K: 25, L: 2, Precision:0.2962962962962963
K: 25, L: 3, Precision:0.48148148148148145
K: 25, L: 4, Precision:0.5555555555555556
K: 25, L: 5, Precision:0.4444444444444444
K: 25, L: 6, Precision:0.5925925925925926
K: 25, L: 7, Precision:0.5925